<a href="https://colab.research.google.com/github/AjayBandari1/Django-Signals/blob/main/AccuKnox_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install django

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 50.1 MB/s eta 0:00:00


In [2]:
import os

if not os.path.exists("/content/myproject"):
    !django-admin startproject myproject
else:
    print("Project already exists, skipping creation.")


In [3]:
# Add the project to the Python path
import sys
sys.path.append("/content/myproject")

In [4]:
# Fix indentation in settings.py to avoid leading spaces/tabs
with open("/content/myproject/myproject/settings.py", "r") as file:
    lines = file.readlines()

with open("/content/myproject/myproject/settings.py", "w") as file:
    for line in lines:
        file.write(line.lstrip())  # Removes leading spaces/tabs

print("✅ Fixed indentation in settings.py")

✅ Fixed indentation in settings.py


In [5]:
# Run migrations to set up the database
!python /content/myproject/manage.py migrate

# Check the Django setup for errors
!python /content/myproject/manage.py check

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

Question 1: By default are django signals executed synchronously or asynchronously? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.

Solution: By default, Django signals are executed synchronously. This means they run immediately when the signal is triggered and do not run in a separate thread or process.


In [13]:

# Import necessary libraries
import os
import django
import time
import threading

# Set the Django project settings
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")

#  Fix for running Django ORM in an async environment (needed in Google Colab)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

#  Initialize Django (must be done before importing models)
django.setup()

#  Now, import Django ORM models after setup
from django.contrib.auth.models import User

#  Delete user first (to avoid duplication errors)
User.objects.filter(username="testuser_sync").delete()

#  Measure user creation time
start_time = time.time()
print(f"🚀 Creating user started at {start_time} in thread: {threading.current_thread().name}")

#  User creation
user = User.objects.create(username="testuser_sync")

end_time = time.time()
print(f"✅ Creating user finished at {end_time} in thread: {threading.current_thread().name}")
print(f"⏱️ Total Execution Time: {end_time - start_time:.4f} seconds")


🚀 Creating user started at 1743227722.781897 in thread: MainThread
Signal running in thread: MainThread
✅ Signal executed for testuser_sync in thread: MainThread
✅ Signal executed AFTER successful transaction for testuser_sync
✅ Creating user finished at 1743227722.7925098 in thread: MainThread
⏱️ Total Execution Time: 0.0106 seconds


Question 2: Do django signals run in the same thread as the caller? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.

Solution: Yes, Django signals run in the same thread as the caller. When a signal is emitted, it executes in the same thread that triggered the signal.

In [14]:
# Create a new user and handle the signal asynchronously
import os
import django
import threading
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth.models import User

# Fix for async environment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Setup Django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
django.setup()

# Ensure user does not already exist
User.objects.filter(username="testuser").delete()

# Define a signal to check the execution thread
@receiver(post_save, sender=User)
def signal_handler(sender, instance, **kwargs):
    print(f"Signal running in thread: {threading.current_thread().name}")

print(f"Main thread: {threading.current_thread().name}")

# Create a new user
user = User.objects.create(username="testuser")


Main thread: MainThread
Signal running in thread: MainThread
✅ Signal executed for testuser in thread: MainThread
✅ Signal executed AFTER successful transaction for testuser
Signal running in thread: MainThread


Question 3: By default do django signals run in the same database transaction as the caller? Please support your answer with a code snippet that conclusively proves your stance. The code does not need to be elegant and production ready, we just need to understand your logic.

Solution : Yes, by default, Django signals run in the same database transaction as the caller. However, if an exception occurs in the caller's transaction, any database changes made in the signal will also be rolled back.

In [10]:
# Wrap user creation in a transaction and raise an exception to test rollback
import os
import django
import threading
from django.db import transaction
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth.models import User

# Fix for async environment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Setup Django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "myproject.settings")
django.setup()

# Ensure user does not already exist
User.objects.filter(username="testuser_txn").delete()

# Define a signal to check if it runs within the transaction
@receiver(post_save, sender=User)
def signal_handler(sender, instance, **kwargs):
    print(f"✅ Signal executed for {instance.username} in thread: {threading.current_thread().name}")

    # Ensure signal runs only after a successful commit
    transaction.on_commit(lambda: print(f"✅ Signal executed AFTER successful transaction for {instance.username}"))

# Wrap user creation in a transaction and raise an exception
try:
    with transaction.atomic():
        print(f"🚀 Creating user in thread: {threading.current_thread().name}")
        user = User.objects.create(username="testuser_txn")
        print(f"⚠️ Raising exception to test rollback...")
        raise Exception("Simulated error after user creation!")
except Exception as e:
    print(f"❌ Exception caught: {e}")

# Check if user exists (It should NOT exist if rollback worked)
user_exists = User.objects.filter(username="testuser_txn").exists()
print(f"🔎 User exists after rollback? {'Yes' if user_exists else 'No'}")


🚀 Creating user in thread: MainThread
Signal running in thread: MainThread
✅ Signal executed for testuser_txn in thread: MainThread
⚠️ Raising exception to test rollback...
❌ Exception caught: Simulated error after user creation!
🔎 User exists after rollback? No


4. Description: You are tasked with creating a Rectangle class with the following requirements:

An instance of the Rectangle class requires length:int and width:int to be initialized.
We can iterate over an instance of the Rectangle class
When an instance of the Rectangle class is iterated over, we first get its length in the format: {'length': <VALUE_OF_LENGTH>} followed by the width {width: <VALUE_OF_WIDTH>}


In [11]:
class Rectangle:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width

    def __iter__(self):
        yield {'length': self.length}
        yield {'width': self.width}

# Example Usage
rect = Rectangle(10, 5)
for attribute in rect:
    print(attribute)

{'length': 10}
{'width': 5}
